# Phase 21: Exit Model (Model 2) - Neural Training
================================================
This notebook trains structural sequential models (TCN and Transformer) to predict if a user is currently abandoning their session.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import sys
from torch.utils.data import Dataset, DataLoader

# Add scripts dir to sys path for architecture imports
sys.path.append(os.path.abspath("../scripts"))
from train_tcn import AbandonmentTCN
from train_transformer import AbandonmentTransformer, ClickstreamDataset

DATA_DIR = "../data/processed"
X_page = np.load(os.path.join(DATA_DIR, "X_page_real.npy"))[:10000] # Subsample for demo
X_dur = np.load(os.path.join(DATA_DIR, "X_dur_real.npy"))[:10000]
y = np.load(os.path.join(DATA_DIR, "y_abandon_real.npy"))[:10000]

print(f"Training on {len(y)} samples.")

Training on 10000 samples.


In [2]:
dataset = ClickstreamDataset(X_page, X_dur, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# num_page_types=4 (View, AddToCart, Transaction, Padding)
model = AbandonmentTCN(num_page_types=4).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

print(f"Training TCN on {device}...")

Training TCN on cpu...


In [3]:
model.train()
for epoch in range(5):
    total_loss = 0
    for page, dur, label in loader:
        page, dur, label = page.to(device), dur.to(device), label.to(device)
        optimizer.zero_grad()
        logits = model(page, dur).squeeze()
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss/len(loader):.4f}")

torch.save(model.state_dict(), "../backend/models/exit_model_tcn.pth")
print("✅ Training complete. Model saved.")

Epoch 1 | Loss: 0.1178
Epoch 2 | Loss: 0.0056
Epoch 3 | Loss: 0.0018
Epoch 4 | Loss: 0.0044
Epoch 5 | Loss: 0.0033
✅ Training complete. Model saved.
